In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
data_path = r'C:\Users\SaWa\Brain Tumors\Brain Tumors Data\brain_tumor_mri\new_dataset\bt_images'

# Construct full paths for the pickle files
training_data_path = os.path.join(data_path, r'C:\Users\SaWa\Brain Tumors\Brain Tumors Data\brain_tumor_mri\new_dataset\training_data.pickle')
labels_path = os.path.join(data_path, r'C:\Users\SaWa\Brain Tumors\Brain Tumors Data\brain_tumor_mri\new_dataset\labels.pickle')
test_data_path = os.path.join(data_path, r'C:\Users\SaWa\Brain Tumors\Brain Tumors Data\test_images-20210704T210303Z-001\test_images')

# Load the pickle files
with open(training_data_path, 'rb') as training_data_file:
    training_data = pickle.load(training_data_file)

with open(labels_path, 'rb') as labels_file:
    labels = pickle.load(labels_file)

In [3]:
from PIL import Image

# Open an image
image_path = r'C:\Users\SaWa\Brain Tumors\Brain Tumors Data\brain_tumor_mri\new_dataset\bt_images\1.jpg'
img = Image.open(image_path)

# Get the dimensions (width and height) of the image
width, height = img.size

print("Image Width:", width)
print("Image Height:", height)

Image Width: 512
Image Height: 512


In [4]:
# Convert to NumPy arrays and normalize.
training_data = np.array(training_data)
labels = np.array(labels)

C:\Users\SaWa\AppData\Local\Temp\ipykernel_17380\1410008418.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


In [5]:
# Convert to NumPy arrays and normalize.
training_data = np.array(training_data)
labels = np.array(labels)

In [6]:
training_data.shape

(3064, 2)

In [7]:
unique_labels = np.unique(labels)
print("Unique Label Values:", unique_labels)

Unique Label Values: [1 2 3]


In [8]:
# Subtract 1 from each label to make them start from 0
adjusted_labels = labels - 1

In [9]:
unique_labels1 = np.unique(adjusted_labels)
print("Unique Label Values:", unique_labels1)

Unique Label Values: [0 1 2]


In [10]:
# Get a list of all files in the folder
file_list = os.listdir(data_path)

In [11]:
# Filter out only image files (e.g., JPG, PNG, etc.)
image_files = [file for file in file_list if file.lower().endswith(('.jpg', '.png', '.jpeg'))]

In [12]:
# List to store processed image data
image_data = []

for image_file in image_files:
    image_path = os.path.join(data_path, image_file)
    img = Image.open(image_path)
    
    # Convert to RGB format if not already
    img = img.convert('RGB')
    
    # Resize the image to the desired dimensions
    img = img.resize((224, 224))
    
    # Convert to NumPy array
    img_array = np.array(img)
    
    # Normalize pixel values and convert to NumPy array
    img_array = np.array(img) / 255.0  
    
    # Append the processed image array to the list
    image_data.append(img_array)

In [13]:
# Convert the list of processed image arrays to a NumPy array
images_array = np.array(image_data)

In [14]:
# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images_array, adjusted_labels, test_size=0.2, random_state=42)

In [15]:
del image_data

In [16]:
# Convert labels to categorical format
num_classes = len(np.unique(adjusted_labels))
train_labels_categorical = to_categorical(train_labels, num_classes=num_classes)
test_labels_categorical = to_categorical(test_labels, num_classes=num_classes)

In [17]:
image_width = 224
image_height = 224
num_channels = 3  # Assuming RGB images
num_classes = 3  # Number of classes in your dataset
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(image_width, image_height, num_channels)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(train_images, train_labels, epochs=15, validation_data=(test_images, test_labels))

Epoch 1/15
77/77 [==============================] - 52s 634ms/step - loss: 2.9776 - accuracy: 0.4782 - val_loss: 1.0341 - val_accuracy: 0.5742
Epoch 2/15
77/77 [==============================] - 21s 272ms/step - loss: 1.0237 - accuracy: 0.4782 - val_loss: 0.9584 - val_accuracy: 0.5742
Epoch 3/15
77/77 [==============================] - 24s 313ms/step - loss: 1.0117 - accuracy: 0.4835 - val_loss: 0.9360 - val_accuracy: 0.4796
Epoch 4/15
77/77 [==============================] - 22s 280ms/step - loss: 0.9916 - accuracy: 0.4790 - val_loss: 0.9614 - val_accuracy: 0.5041
Epoch 5/15
77/77 [==============================] - 22s 287ms/step - loss: 0.9859 - accuracy: 0.4843 - val_loss: 0.8970 - val_accuracy: 0.5546
Epoch 6/15
77/77 [==============================] - 21s 278ms/step - loss: 0.9616 - accuracy: 0.4986 - val_loss: 0.8874 - val_accuracy: 0.4551
Epoch 7/15
77/77 [==============================] - 21s 276ms/step - loss: 0.9488 - accuracy: 0.5014 - val_loss: 0.8566 - val_accuracy: 0.4698

In [20]:
# Evaluate the model
evaluation = model.evaluate(test_images, test_labels)

20/20 [==============================] - 1s 34ms/step - loss: 0.7979 - accuracy: 0.5106
